# CNSH - Backtracking 이해

## 1. 문제 개요

**해밀토니안 경로 (Hamiltonian Path)**
- 그래프의 모든 노드를 정확히 한 번씩만 방문하는 경로
- 이 문제: 모든 해밀토니안 경로 중 **가중치 합이 최소**인 경로 찾기

## 2. 원본 코드

파일: `codes/cnsh-sample/1413-LJ-077.py`

In [1]:
def f(x,y,z,d):
    global V, visited, ok , ans
    if ok : return
    if y == V :
        if ans > d : ans = d
        return
    visited[x] = True
    for u, w in M[x]:
        if visited[u] == False:
            z.append(u)
            f(u, y+1, z, d+w)
            z.pop()
    visited[x] = False
    return

M = [[] for i in range(11)]
ans = 123456789987654321
visited = [0 for i in range(11)]
ok = False
V, E = map(int, input().split())

for i in range(E):
    u, v, w = map(int, input().split())
    M[u].append((v,w))
    M[v].append((u,w))

for v in range(0, V+1):
    f(v, 1, [v],0)

print(ans)

KeyboardInterrupt: Interrupted by user

## 3. 함수 파라미터 의미

```python
def f(x, y, z, d):
```

| 파라미터 | 의미 | 타입 | 변화 |
|---------|------|------|-----|
| `x` | 현재 방문 중인 노드 | int | 재귀마다 다름 |
| `y` | 지금까지 방문한 노드 개수 | int | 1 → 2 → 3 → ... → V |
| `z` | 현재까지의 경로 리스트 | list | append/pop으로 관리 |
| `d` | 현재까지의 가중치 합계 | int | 누적 증가 |

## 4. 전역 변수 vs 파라미터

### 전역 변수: 모든 재귀 호출이 공유

```python
M = [[] for i in range(11)]     # 그래프 구조 (불변)
V = 0                            # 노드 개수 (불변)
visited = [False] * 11          # 방문 상태 (백트래킹으로 변경)
ans = 123456789987654321        # 최솟값 (모든 경로 탐색 후 업데이트)
```

### 파라미터: 재귀 깊이마다 다른 값

```python
x - 현재 노드 위치
y - 현재 재귀 깊이
z - 현재 경로 (하지만 사실상 공유...)
d - 누적 비용
```

## 5. 중요한 질문: y 파라미터가 필요한가?

### 의문점

```python
y == len(z)  # 항상 참
```

항상 `y`와 `len(z)`가 같다면, 하나만 있어도 충분하지 않을까?

### 검증

```python
# 초기 호출
f(v, 1, [v], 0)           # y=1, len(z)=1 ✓

# 재귀 호출
z.append(u)               # len(z) = y+1
f(u, y+1, z, d+w)         # y=y+1, len(z)=y+1 ✓
```

### y를 사용하는 이유

1. **성능**: `len(z)` 함수 호출 오버헤드 회피 (미세)
2. **명시성**: 함수 시그니처에서 "방문 개수"가 중요 파라미터임을 명확히
3. **불변성**: 정수는 불변 → 재귀 시 자동 복원, 실수 가능성 감소
4. **관습**: 백트래킹 알고리즘 교육 표준 패턴

### 대안: y 제거

```python
def f(x, z, d):
    if len(z) == V:  # y 대신 len(z) 사용
        update_ans(d)
        return
    # ...
```

**결론**: 기능적으로는 `len(z)` 사용 가능하지만, 백트래킹 패턴에서는 깊이/개수를 명시적 파라미터로 관리하는 것이 일반적

## 6. 설계의 일관성 문제

### 어정쩡한 점

```python
# z는 파라미터인데...
def f(x, y, z, d):
    z.append(u)      # 원본 리스트 수정
    f(u, y+1, z, d+w)  # 같은 z 객체 전달
    z.pop()          # 원본 리스트 수정
```

**문제점:**
- `z`는 파라미터로 전달되지만 **사실상 전역처럼 공유**됨
- "깊이마다 다른 값"이 아니라 "하나의 리스트를 수동 관리"
- `y`는 `len(z)`와 항상 같은데 둘 다 관리

### 일관성 있게 만들려면

**옵션 1: 전역 제대로 쓰기**
```python
# z, y도 전역으로
path = []
def f(x, d):
    if len(path) == V:
        # ...
```

**옵션 2: 지역화 제대로 하기**
```python
# visited도 복사본 전달
def f(x, y, z, d, visited_copy):
    # ...
```

### 왜 이렇게 설계했을까?

많은 알고리즘 교재에서 이런 **백트래킹 템플릿**을 표준으로 가르침:

```python
def backtrack(위치, 깊이, 경로, 누적값):
    if 종료조건(깊이):
        결과처리(누적값)
        return
    
    for 다음위치 in 후보들:
        경로.추가(다음위치)
        backtrack(다음위치, 깊이+1, 경로, 누적값+값)
        경로.제거()  # 백트래킹
```

**결론**: 최적 설계는 아니지만, 재귀와 백트래킹 학습을 위한 전통적인 패턴

## 7. 리팩토링 버전: 작은 함수로 분해

기존 구조를 유지하면서 작은 함수로 분해한 버전

In [ ]:
# 전역 변수 선언
M = []          # 그래프 (인접 리스트)
ans = 0         # 최소 비용
visited = []    # 방문 여부
V, E = 0, 0     # 노드 개수, 간선 개수


def init_graph():
    """그래프와 방문 배열을 초기화한다"""
    global M, visited, ans
    M = [[] for i in range(11)]
    visited = [False for i in range(11)]
    ans = 123456789987654321


def add_edge(u, v, w):
    """그래프에 양방향 간선을 추가한다"""
    global M
    M[u].append((v, w))
    M[v].append((u, w))


def read_input():
    """그래프 정보를 입력받는다"""
    global V, E
    V, E = map(int, input().split())

    for i in range(E):
        u, v, w = map(int, input().split())
        add_edge(u, v, w)


def update_ans(d):
    """현재 비용 d가 최솟값보다 작으면 ans를 업데이트한다"""
    global ans
    if ans > d:
        ans = d


def f(x, y, z, d):
    """
    해밀토니안 경로를 재귀적으로 탐색한다

    x: 현재 방문 중인 노드
    y: 지금까지 방문한 노드 개수
    z: 현재까지의 경로 리스트
    d: 현재까지의 비용 합계
    """
    global V, M, visited

    # 모든 노드를 방문했으면 최솟값 업데이트
    if y == V:
        update_ans(d)
        return

    # 현재 노드 방문 표시
    visited[x] = True

    # 인접한 노드들을 탐색
    for u, w in M[x]:
        # M에 간선을 추가할 때 tuple로 저장하였으므로
        # u, w = (번호, 가중치)를 사용해 M에 저장되어 있는 튜플 값을 분해 할당
        if visited[u] == False:
            z.append(u)              # 경로에 추가
            f(u, y+1, z, d+w)        # 재귀 호출
            z.pop()                  # 백트래킹: 경로에서 제거

    # 백트래킹: 방문 표시 해제
    visited[x] = False


def find_min_cost():
    """모든 시작점에서 해밀토니안 경로를 탐색한다"""
    global V

    for v in range(V + 1):
        f(v, 1, [v], 0)


def main():
    """메인 실행 함수"""
    global ans

    # 초기화
    init_graph()

    # 입력 받기
    read_input()

    # 최소 비용 해밀토니안 경로 찾기
    find_min_cost()

    # 결과 출력
    print(ans)


if __name__ == "__main__":
    main()

### 함수 분해 구조

1. **`init_graph()`** - 그래프와 배열 초기화
2. **`add_edge(u, v, w)`** - 간선 추가
3. **`read_input()`** - 입력 받기
4. **`update_ans(d)`** - 최솟값 업데이트
5. **`f(x, y, z, d)`** - 핵심 재귀 탐색
6. **`find_min_cost()`** - 모든 시작점에서 탐색
7. **`main()`** - 전체 실행 흐름

**장점:**
- 각 함수가 명확한 역할 하나씩 담당
- 작은 함수를 조합하여 전체 알고리즘 구성
- 디버깅과 이해가 쉬워짐

## 8. 핵심 개념: 백트래킹 (Backtracking)

### 백트래킹 패턴

1. **선택**: 가능한 선택지 중 하나를 선택
2. **진행**: 선택을 바탕으로 다음 단계로 진행 (재귀)
3. **되돌리기**: 해를 찾지 못하면 이전 상태로 되돌림
4. **반복**: 모든 가능성 탐색

### 이 코드에서의 백트래킹

```python
visited[x] = True        # 1. 선택: 노드 방문
z.append(u)              # 

f(u, y+1, z, d+w)        # 2. 진행: 다음 노드로 재귀

z.pop()                  # 3. 되돌리기: 경로에서 제거
visited[x] = False       #    방문 취소
```

**핵심**: `append`/`pop`, `True`/`False`를 **쌍으로 관리**

### 해밀토니안 경로 탐색 과정 예시

```
그래프: 0-1-2-3 (가중치 생략)

0 → 1 → 2 → 3 (완성: 최솟값 갱신)
    ↓ (백트래킹)
0 → 1 → 3 → 2 (완성: 최솟값 갱신)
    ↓ (백트래킹)
0 → 2 → 1 → 3 (완성: 최솟값 갱신)
    ↓ (백트래킹)
...
```

## 9. 동작 과정 시각화

### 간단한 그래프 예시

```
노드: 3개 (V=3)
간선: 0-1(w=1), 1-2(w=2), 0-2(w=5)

  0
 /|\  
1 5 1
|   |
2---2
```

### 탐색 과정

```python
f(0, 1, [0], 0)                    # 시작: 0번 노드
  visited[0] = True
  
  f(1, 2, [0,1], 1)                # 0→1 (비용 1)
    visited[1] = True
    
    f(2, 3, [0,1,2], 3)            # 1→2 (비용 2, 누적 3)
      visited[2] = True
      y==V → ans = min(ans, 3)     # 완성! 최솟값 갱신
      visited[2] = False           # 백트래킹
    
    visited[1] = False             # 백트래킹
  
  f(2, 2, [0,2], 5)                # 0→2 (비용 5)
    visited[2] = True
    
    f(1, 3, [0,2,1], 6)            # 2→1 (비용 1, 누적 6)
      visited[1] = True
      y==V → ans = min(ans, 6)     # 완성! (3 < 6이므로 갱신 안 됨)
      visited[1] = False           # 백트래킹
    
    visited[2] = False             # 백트래킹
  
  visited[0] = False               # 백트래킹
```

**결과**: `ans = 3` (경로: 0→1→2)

## 10. 학습 포인트

### 1. 재귀 함수의 파라미터 설계
- 어떤 값을 전역으로, 어떤 값을 파라미터로 전달할지
- 각 선택의 trade-off 이해

### 2. 백트래킹 패턴
- 선택 → 진행 → 되돌리기 구조
- `append`/`pop`, `True`/`False` 쌍으로 관리
- 상태 변경과 복원의 대칭성

### 3. 효율성 vs 명확성
- `y` 파라미터: 꼭 필요하지 않지만 명시성을 위해 사용
- 실무에서는 더 깔끔한 설계가 가능하지만, 학습 단계에서는 전통적 패턴 따름

### 4. 전역 변수의 활용
- 모든 재귀 호출이 공유하는 데이터 (그래프 구조, 결과값 등)
- 백트래킹으로 변경되지만 같은 객체를 공유 (visited 배열)

### 5. 최솟값 찾기 패턴
- 큰 초기값 설정: `ans = 123456789987654321`
- 비교 연산: `if ans > d : ans = d`
- 최댓값 찾기는 반대로